In [1]:
import pandas as pd
import plotly   
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State 
import requests
from datetime import datetime

app = dash.Dash(__name__)

In [2]:
COVID = pd.read_csv('CleanedCOVIDdata.csv')


In [3]:
tabs_styles = {
    'height': '180px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px'
    
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#f1f1f1',
    'color': 'Black',
    'padding': '6px',
    'fontWeight': 'bold'
}

app.layout = html.Div(
    [
        html.H1("COVID-19 App for Data Science", style={"textAlign": "center"}),
        dcc.Store(id="aggregate_data"),
        # empty Div to trigger javascript file for graph resizing
        html.Div(id="output-clientside"),
        html.Div(
            [
                html.Br(),
        dcc.Tabs(id="tabs", children=[
        # Defining the layout of the first Tab
        dcc.Tab(label='COVID Trends', style=tab_style, selected_style=tab_selected_style, children=[
        html.Div(
            [
                html.Div(
                    [
                        html.P("Apply to all filter:", className="control_label",style={'textAlign': 'center'}),
                        dcc.Checklist(
                            id="Filterbydate",
                            options=[
                                {"label": "By day", "value": "D"},
                                {"label": "By week", "value": "W-MON"},
                                {"label": "By month", "value": "new_deaths"},
                            ],
                            value="D",
                            labelStyle={"display": "inline-block",'textAlign': 'center'},
                            style={'textAlign': 'center'},
                            className="dcc_control",
                        ),
                       
                        html.P("Filter by Continent:", className="control_label"),
                        dcc.RadioItems(
                            id="RadioFilterbyContinent",
                            options=[
                                {"label": "All", "value": "all"},
                                {"label": "New Cases", "value": "productive"},
                                {"label": "New Dealth", "value": "custom"},
                            ],
                            value="D",
                            labelStyle={"display": "inline-block"},
                            
                            className="dcc_control",
                        ),
                        dcc.Dropdown(
                            id="DropFilterbyContinent",
                            options=[{'label':x, 'value':x} for x in COVID.sort_values('continent')['continent'].unique()],
                            multi=True,
                            #value=list(WELL_STATUSES.keys()),
                            placeholder='Choose Continent...',
                            className="dcc_control",
                        ),
                        
                        html.P("Filter by Country:", className="control_label"),
                        dcc.RadioItems(
                            id="RadioFilterbyCountry",
                            options=[
                                {"label": "All ", "value": "all"},
                                {"label": "New Cases", "value": "productive"},
                                {"label": "New Dealth", "value": "custom"},
                            ],
                            value="All",
                            labelStyle={"display": "inline-block"},
                            className="dcc_control",
                        ),
                        dcc.Dropdown(
                            id="dropFilterbyCountry",
                            #options=[{'label':x, 'value':x} for x in COVID.sort_values('Country')['Country'].unique()],                            multi=True,
                            placeholder='Choose Country...',
                          #  value=list(WELL_TYPES.keys()),
                            className="dcc_control",
                        ),
                        html.P("Filter by State:", className="control_label"),
                        dcc.RadioItems(
                            id="RadioFilterbystate",
                            options=[
                                {"label": "All ", "value": "all"},
                                {"label": "New Cases", "value": "productive"},
                                {"label": "New Dealth", "value": "custom"},
                            ],
                            value="All",
                            labelStyle={"display": "inline-block"},
                            className="dcc_control",
                        ),
                        dcc.Dropdown(
                            id="dropFilterbystate",
                            options=[{'label':x, 'value':x} for x in COVID.sort_values('State')['State'].unique()],                            
                            placeholder='Choose State...',
                            multi=True,
                          #  value=list(WELL_TYPES.keys()),
                            className="dcc_control",
                        ),
                    ],
                    className="pretty_container four columns",
                    id="cross-filter-options",
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                html.Div(
                                    [html.H6(id="Confirmed_text"), html.P("Confirmed Cases")],
                                    id="Confirmed",
                                    className="mini_container",
                                ),
                                html.Div(
                                    [html.H6(id="Dealth_text"), html.P("Dealth Cases")],
                                    id="Dealth",
                                    className="mini_container",
                                ),
                                html.Div(
                                    [html.H6(id="Recovered_text"), html.P("Recovered Cases")],
                                    id="Recovered",
                                    className="mini_container",
                                ),
                                html.Div(
                                    [html.H6(id="Hospitalized_text"), html.P("Hospitalized Cases")],
                                    id="Hospitalized",
                                    className="mini_container",
                                ),
                            ],
                            id="info-container",
                            className="row container-display",
                        ),
                        html.Div(
                            [dcc.Graph(id="count_graph")],
                            id="countGraphContainer",
                            className="pretty_container",
                        ),
                    ],
                    id="right-column",
                    className="eight columns",
                ),
                
                
                ],className="row flex-display",
                ),
                html.Div(
                    [
                    html.Div(
                        [dcc.Graph(id="main_graph")],
                        className="pretty_container seven columns",
                    ),
                    html.Div(
                    [dcc.Graph(id="individual_graph")],
                    className="pretty_container five columns",
                ),
                ],
                className="row flex-display",
                ),
                html.Div(
                [
                html.Div(
                    [dcc.Graph(id="pie_graph")],
                    className="pretty_container seven columns",
                ),
                html.Div(
                    [dcc.Graph(id="aggregate_graph")],
                    className="pretty_container five columns",
                ),
                ],
                className="row flex-display",
                ),
                ],),
            
            # Defining the layout of the second tab
        dcc.Tab(label='Machine Learning', style=tab_style, selected_style=tab_selected_style,children=[
            html.H1("Daily trend", 
                    style={"textAlign": "center"})
        ],),
            
            
            
                ],),],),],
    id="mainContainer",
    style={"display": "flex", "flex-direction": "column"},
)

In [4]:
@app.callback(
    Output('dropFilterbyCountry', 'options'),
    [Input('DropFilterbyContinent', 'value')])
def set_metrics_options(selected_continent):
        if selected_continent.isnull():
        dff1 = COVID.loc[COVID['continent'].isin(selected_continent)]
        dff2 = dff1['Country'].unique()
        dff3 = [{'label': i, 'value': i} for i in dff2]
        print(dff3)
        return dff3

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Aug/2020 15:34:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2020 15:34:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2020 15:34:49] "GET /_dash-dependencies HTTP/1.1" 200 -


None
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1031, in dispatch
    response.

127.0.0.1 - - [04/Aug/2020 15:34:49] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [04/Aug/2020 15:35:07] "POST /_dash-update-component HTTP/1.1" 200 -


['Asia']
[{'label': 'Afghanistan', 'value': 'Afghanistan'}, {'label': 'Armenia', 'value': 'Armenia'}, {'label': 'Azerbaijan', 'value': 'Azerbaijan'}, {'label': 'Bahrain', 'value': 'Bahrain'}, {'label': 'Cambodia', 'value': 'Cambodia'}, {'label': 'China', 'value': 'China'}, {'label': 'Georgia', 'value': 'Georgia'}, {'label': 'India', 'value': 'India'}, {'label': 'Indonesia', 'value': 'Indonesia'}, {'label': 'Iran', 'value': 'Iran'}, {'label': 'Iraq', 'value': 'Iraq'}, {'label': 'Israel', 'value': 'Israel'}, {'label': 'Japan', 'value': 'Japan'}, {'label': 'Kuwait', 'value': 'Kuwait'}, {'label': 'Lebanon', 'value': 'Lebanon'}, {'label': 'Malaysia', 'value': 'Malaysia'}, {'label': 'Nepal', 'value': 'Nepal'}, {'label': 'Oman', 'value': 'Oman'}, {'label': 'Pakistan', 'value': 'Pakistan'}, {'label': 'Philippines', 'value': 'Philippines'}, {'label': 'Qatar', 'value': 'Qatar'}, {'label': 'Singapore', 'value': 'Singapore'}, {'label': 'Sri Lanka', 'value': 'Sri Lanka'}, {'label': 'Thailand', 'val

127.0.0.1 - - [04/Aug/2020 15:36:44] "POST /_dash-update-component HTTP/1.1" 200 -


[]
[]
None
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1031, in dispatch
    res

127.0.0.1 - - [04/Aug/2020 17:10:15] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [04/Aug/2020 17:10:20] "POST /_dash-update-component HTTP/1.1" 200 -


['Europe']
[{'label': 'Austria', 'value': 'Austria'}, {'label': 'Belarus', 'value': 'Belarus'}, {'label': 'Belgium', 'value': 'Belgium'}, {'label': 'Croatia', 'value': 'Croatia'}, {'label': 'Denmark', 'value': 'Denmark'}, {'label': 'Estonia', 'value': 'Estonia'}, {'label': 'Finland', 'value': 'Finland'}, {'label': 'France', 'value': 'France'}, {'label': 'Germany', 'value': 'Germany'}, {'label': 'Greece', 'value': 'Greece'}, {'label': 'Iceland', 'value': 'Iceland'}, {'label': 'Ireland', 'value': 'Ireland'}, {'label': 'Italy', 'value': 'Italy'}, {'label': 'Lithuania', 'value': 'Lithuania'}, {'label': 'Luxembourg', 'value': 'Luxembourg'}, {'label': 'Monaco', 'value': 'Monaco'}, {'label': 'Netherlands', 'value': 'Netherlands'}, {'label': 'Norway', 'value': 'Norway'}, {'label': 'Romania', 'value': 'Romania'}, {'label': 'Russia', 'value': 'Russia'}, {'label': 'San Marino', 'value': 'San Marino'}, {'label': 'Spain', 'value': 'Spain'}, {'label': 'Sweden', 'value': 'Sweden'}, {'label': 'Switzer

In [ ]:
#dff1 = COVID.loc[COVID['continent'].isin(['Asia'])]
#dff2 = dff1['Country'].unique()
#dff3 = [{'label': i, 'value': i} for i in dff2]
#print(dff3)